In [1]:
#Get GG historical results from wikipedia instead of IMDb

import pandas as pd

In [47]:
#Amazing, extracts tables from an html page
res = pd.read_html('https://en.wikipedia.org/wiki/Golden_Globe_Award_for_Best_Actor_%E2%80%93_Television_Series_Drama')

In [56]:
actor_drama = pd.concat([t[['Year','Program','Actor']] for t in res[3:9]])
actor_drama['Year'] = actor_drama['Year'].apply(lambda v: int(v[:4])+1)  #wikipedia calls the ceremonies by year noms announced
actor_drama = actor_drama[actor_drama.Program.notnull()]
actor_drama['category'] = 'Actor Drama'
#can't tell who won exactly, but it is usually just the first in the year group
est_winners = pd.DataFrame(actor_drama.groupby('Year').Actor.first()).reset_index()
est_winners['won'] = 1
actor_drama = actor_drama.merge(est_winners, how='left', on=['Year','Actor'])
actor_drama.loc[actor_drama.won.isnull(), 'won'] = 0
actor_drama['won'] = actor_drama.won.astype(int)
actor_drama.columns = ['year','show','performer','category','won']

In [88]:
#and check show categories format
res = pd.read_html('https://en.wikipedia.org/wiki/Golden_Globe_Award_for_Best_Television_Series_%E2%80%93_Drama')
#res[2]  #want 2-7, Year, Program

In [118]:
full_res = []
for url, col_name, category_name in zip(['https://en.wikipedia.org/wiki/Golden_Globe_Award_for_Best_Actress_%E2%80%93_Television_Series_Drama',
                                         'https://en.wikipedia.org/wiki/Golden_Globe_Award_for_Best_Actor_%E2%80%93_Television_Series_Drama',
                                         'https://en.wikipedia.org/wiki/Golden_Globe_Award_for_Best_Actress_%E2%80%93_Television_Series_Musical_or_Comedy',
                                         'https://en.wikipedia.org/wiki/Golden_Globe_Award_for_Best_Actor_%E2%80%93_Television_Series_Musical_or_Comedy'],
                                        ['Actress','Actor','Actor','Actor'],
                                        ['Actress Drama','Actor Drama','Actress Comedy','Actor Comedy']):
    res = pd.read_html(url)
    res_df = pd.concat([t[['Year','Program',col_name]] for t in res[3:9]])
    res_df['Year'] = res_df['Year'].apply(lambda v: int(v[:4])+1)
    res_df = res_df[res_df.Program.notnull()]
    res_df['category'] = category_name
    #can't tell who won exactly, but it is usually just the first in the year group
    est_winners = pd.DataFrame(res_df.groupby('Year')[col_name].first()).reset_index()
    est_winners['won'] = 1
    res_df = res_df.merge(est_winners, how='left', on=['Year',col_name])
    res_df.loc[res_df.won.isnull(), 'won'] = 0
    res_df['won'] = res_df.won.astype(int)
    res_df.columns = ['year','show','performer','category','won']
    #fill in joint winners manually
    if category_name == 'Actor Drama':
        res_df.loc[(res_df.year==1976) & (res_df.performer=='Telly Savalas'), 'won'] = 1
    elif category_name == 'Actor Comedy':
        res_df.loc[(res_df.year==1978) & (res_df.performer=='Henry Winkler'), 'won'] = 1
        res_df.loc[(res_df.year==1989) & (res_df.performer=='Judd Hirsch'), 'won'] = 1
        res_df.loc[(res_df.year==1989) & (res_df.performer=='Richard Mulligan'), 'won'] = 1
    elif category_name == 'Actress Drama':
        res_df.loc[(res_df.year==1982) & (res_df.performer=='Barbara Bel Geddes'), 'won'] = 1
        res_df.loc[(res_df.year==1991) & (res_df.performer=='Patricia Wettig'), 'won'] = 1
    elif category_name == 'Actress Comedy':
        res_df.loc[(res_df.year==1974) & (res_df.performer=='Jean Stapleton'), 'won'] = 1
        res_df.loc[(res_df.year==1986) & (res_df.performer=='Cybill Shepherd'), 'won'] = 1
    
    full_res.append(res_df[['category','year','show','performer','won']])

In [119]:
for url, category_name in zip(['https://en.wikipedia.org/wiki/Golden_Globe_Award_for_Best_Television_Series_%E2%80%93_Drama',
                               'https://en.wikipedia.org/wiki/Golden_Globe_Award_for_Best_Television_Series_%E2%80%93_Musical_or_Comedy'],
                              ['Show Drama','Show Comedy']):
    res = pd.read_html(url)
    if category_name == 'Show Drama':
        res_df = pd.concat([t[['Year','Program']] for t in res[2:8]])
    else:
        res_df = pd.concat([t[['Year','Program']] for t in res[1:7]])
    res_df['Year'] = res_df['Year'].apply(lambda y: y+1)  #already an integer
    res_df = res_df[res_df.Program.notnull()]
    res_df['category'] = category_name
    #can't tell who won exactly, but it is usually just the first in the year group
    est_winners = pd.DataFrame(res_df.groupby('Year')['Program'].first()).reset_index()
    est_winners['won'] = 1
    res_df = res_df.merge(est_winners, how='left', on=['Year','Program'])
    res_df.loc[res_df.won.isnull(), 'won'] = 0
    res_df['won'] = res_df.won.astype(int)
    res_df['performer'] = None
    res_df.columns = ['year','show','category','won','performer']
    #fill in joint winners manually
    if category_name == 'Show Comedy':
        res_df.loc[(res_df.year==1980) & (res_df.show=='Taxi'), 'won'] = 1
    
    full_res.append(res_df[['category','year','show','performer','won']])

In [120]:
full_res = pd.concat(full_res)    

In [ ]:
#a few entries are missing from the summary pages
full_res = pd.concat([full_res,
                      pd.DataFrame({'category': ['Actress Drama'],
                                    'year': [1973],
                                    'show': ['The Delphi Bureau'],
                                    'performer': ['Anne Jeffreys'],
                                    'won': [0]
                                   })
                     ])


In [ ]:
full_res.groupby('year').won.sum()  #check mostly 6s with 7s and one 8 in years manually altered

In [125]:
full_res[(full_res.year >= 1972) & (full_res.year < 2021)].sort_values(['year','category']).to_csv('/home/david/projects/gglobes_vis/data/gglobes_data_wikipedia_1972to2020.csv.csv', index=None)